In [2]:
#Import preprocessing libraries
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline as ImblearnPipeline
from sklearn.metrics import accuracy_score

In [3]:
#Loading the datasets
df = pd.read_csv('eda.csv')

In [4]:
#Dropping the potential column as a players potential cannot be known till he excels in a certain position as well as the interntional 
#reputation column as it should not be a determinant for a players best position
df = df.drop(columns = ['Potential', 'International Reputation'])

In [5]:
df['Weak Foot Rating']

0        4
1        4
2        4
3        5
4        4
        ..
18534    3
18535    3
18536    3
18537    3
18538    3
Name: Weak Foot Rating, Length: 18539, dtype: int64

In [6]:
#Info about datasets
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18539 entries, 0 to 18538
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Overall             18539 non-null  int64 
 1   Age                 18539 non-null  int64 
 2   Height(in cm)       18539 non-null  int64 
 3   Weight(in kg)       18539 non-null  int64 
 4   Weak Foot Rating    18539 non-null  int64 
 5   Skill Moves         18539 non-null  int64 
 6   Pace Total          18539 non-null  int64 
 7   Shooting Total      18539 non-null  int64 
 8   Passing Total       18539 non-null  int64 
 9   Dribbling Total     18539 non-null  int64 
 10  Defending Total     18539 non-null  int64 
 11  Physicality Total   18539 non-null  int64 
 12  Crossing            18539 non-null  int64 
 13  Finishing           18539 non-null  int64 
 14  Heading Accuracy    18539 non-null  int64 
 15  Short Passing       18539 non-null  int64 
 16  Curve               18

## Preprocessing

In [7]:
#Copying my datsframe
data =df.copy()

In [8]:
#Splitting the features and target columns
X = data.drop('Best Position', axis = 1)
y = data['Best Position']
data.columns

Index(['Overall', 'Age', 'Height(in cm)', 'Weight(in kg)', 'Weak Foot Rating',
       'Skill Moves', 'Pace Total', 'Shooting Total', 'Passing Total',
       'Dribbling Total', 'Defending Total', 'Physicality Total', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Curve',
       'Acceleration', 'Agility', 'Reactions', 'Balance', 'Shot Power',
       'Jumping', 'Stamina', 'Strength', 'Aggression', 'Vision', 'Penalties',
       'Composure', ' GoalkeeperKicking', 'Nationality', 'Preferred Foot',
       'Best Position'],
      dtype='object')

In [9]:
#Splitting numerical and categorical features
num_features = ['Overall', 'Age', 'Height(in cm)', 'Weight(in kg)', 'Weak Foot Rating',
       'Skill Moves', 'Pace Total', 'Shooting Total', 'Passing Total',
       'Dribbling Total', 'Defending Total', 'Physicality Total', 'Crossing',
       'Finishing', 'Heading Accuracy', 'Short Passing', 'Curve',
       'Acceleration', 'Agility', 'Reactions', 'Balance', 'Shot Power',
       'Jumping', 'Stamina', 'Strength', 'Aggression', 'Vision', 'Penalties',
       'Composure', ' GoalkeeperKicking']
cat_features  = ['Nationality', 'Preferred Foot']

In [10]:
#Creating a pipeline for transforming both categorical and numerical features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [11]:
preprocessor = ColumnTransformer(transformers= [
    ('num', numeric_transformer, num_features),
    ('cat', categorical_transformer, cat_features)
    ],
      remainder = 'passthrough'                       
    )

In [15]:
#Classifying certain positions into 4 main categories Defender, Midfielder, Forward and Goalkeeper
y.value_counts()
defends = ['CB', 'RB', 'LB', 'RWB', 'LWB'] #Defenders
midfield = ['CAM', 'RM', 'CDM', 'CM','LM'] #Midfielders
forward = ['ST', 'RW', 'LW', 'CF'] #Forwards
keeper = ['GK'] #Goalkeeper
y_precise = []
for i in y:
    if i in defends:
        y_precise.append(i.replace(i, 'Defender'))
    elif i in midfield:
        y_precise.append(i.replace(i, 'Midfielder'))
    elif i in forward:
        y_precise.append(i.replace(i, 'Forward'))
    else:
        y_precise.append(i.replace(i, 'Goalkeeper'))

In [21]:
#NEw target column
y_prec = pd.Series(y_precise, name = 'Best Position')

In [23]:
#Total number for each of the four player position categories
y_prec.value_counts()

Midfielder    7058
Defender      6273
Forward       3147
Goalkeeper    2061
Name: Best Position, dtype: int64

In [25]:
#Transforming my target column for model training
target = {'Midfielder':0,
         'Defender':1,
         'Forward':2,
         'Goalkeeper': 3,
         }
y_trans = []
for i in y_prec:
    y_trans.append(target[i])

In [26]:
#target column for model training
y_tran = pd.Series(y_trans, name = 'Best Position')

## Model Training

In [28]:
#Importing models for training data
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.svm import SVC

In [29]:
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XG Boost': XGBClassifier(),
    'SVM': SVC(kernel='linear')
}

list(models.keys())

['Logistic Regression', 'Decision Tree', 'Random Forest', 'XG Boost', 'SVM']

In [30]:
for i,classifier in enumerate(list(models.values())):
    
    resampling_pipeline = ImblearnPipeline([
    ('preprocessor', preprocessor),
    ('classifier', classifier)
    ])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_tran, test_size=0.3, random_state=42)
    
    model = resampling_pipeline
    model.fit(X_train, y_train) #train model
    
    #Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    #Training set performance
    model_train_accuracy = accuracy_score(y_train, y_train_pred)
    model_train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    model_train_precision = precision_score(y_train, y_train_pred, average='weighted')
    model_train_recall = recall_score(y_train, y_train_pred, average='weighted')
    #model_train_rocauc_score = roc_auc_score(y_train, y_train_pred, average='weighted', multi_class='ovr')

    
    #Test set performance
    model_test_accuracy = accuracy_score(y_test, y_test_pred)
    model_test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    model_test_precision = precision_score(y_test, y_test_pred, average='weighted')
    model_test_recall = recall_score(y_test, y_test_pred, average='weighted')
    #model_test_rocauc_score = roc_auc_score(y_test, y_test_pred, average='weighted')
                                            
    print(list(models.keys())[i])
    
    print('Model peformance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print("- F1 score: {:.4f}".format(model_train_f1))
                                            
    print("- Precision: {:.4f}".format(model_train_precision))
    print("- Recall: {:.4f}".format(model_train_recall))
    #print("- Roc Auc Score: {:.4f}".format(model_train_rocauc_score))
                                            
                                            
                                            
    print('---------------------------------------')
                                            
    print('Model performance for Test set')
    print("- Accuracy: {:.4f}".format(model_test_accuracy))
    print("- F1 score: {:.4f}".format(model_test_f1))
    print("- Precision: {:.4f}".format(model_test_precision))
    print("- Recall: {:.4f}".format(model_test_recall))
    #print("- Roc Auc Score: {:.4f}".format(model_test_rocauc_score))
                                            
                                            
    print('='*35)
    print('\n')

C:\Users\Dipeolu Ayomide\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression
Model peformance for Training set
- Accuracy: 0.9224
- F1 score: 0.9224
- Precision: 0.9225
- Recall: 0.9224
---------------------------------------
Model performance for Test set
- Accuracy: 0.9157
- F1 score: 0.9156
- Precision: 0.9162
- Recall: 0.9157


Decision Tree
Model peformance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
---------------------------------------
Model performance for Test set
- Accuracy: 0.8637
- F1 score: 0.8638
- Precision: 0.8639
- Recall: 0.8637


Random Forest
Model peformance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
---------------------------------------
Model performance for Test set
- Accuracy: 0.9047
- F1 score: 0.9046
- Precision: 0.9084
- Recall: 0.9047


XG Boost
Model peformance for Training set
- Accuracy: 1.0000
- F1 score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
---------------------------------------
Model performance for T

## Hyperparameter Tuning

In [31]:
#Hyper parameter training for XGboost

#Parameters to be used for tuning XGboost model
params = {
    "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "max_depth"        : [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight" : [1, 3, 5, 7 ],
    "gamma"            : [0.0, 0.1, 0.2 , 0.3, 0.4],
    "colsample_bytree" : [0.3, 0.4, 0.5 , 0.7]
}

In [32]:
#Importing the randomized search that helps to select the best parameters
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [33]:
classifier=XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
boss_tuning = ImblearnPipeline([
    ('preprocessor', preprocessor),
    ('tuning', random_search)
])
boss_tuning.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\Dipeolu Ayomide\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Overall', 'Age',
                                                   'Height(in cm)',
                                                   'Weight(in kg)',
                                                   'Weak Foot Rating',
                                                   'Skill Moves', 'Pace Total',
                                                   'Shooting Total',
                                                   'Passing Total',
                                                   'Dribbling Total',
                                                   'Defending Total',
                                                   'Physicality Total',
                                                   'Crossing', 'Finishing',
                                                   'Heading Accur...
                                                            monotone_constraints=None,
                                                            n_estimators=100,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            predictor=None,
                                                            random_state=None, ...),
                                    n_iter=5, n_jobs=-1,
                                    param_distributions={'colsample_bytree': [0.3,
                                                                              0.4,
                                                                              0.5,
                                                                              0.7],
                                                         'gamma': [0.0, 0.1,
                                                                   0.2, 0.3,
                                                                   0.4],
                                                         'learning_rate': [0.05,
                                                                           0.1,
                                                                           0.15,
                                                                           0.2,
                                                                           0.25,
                                                                           0.3],
                                                         'max_depth': [3, 4, 5,
                                                                       6, 8, 10,
                                                                       12, 15],
                                                         'min_child_weight': [1,
                                                                              3,
                                                                              5,
                                                                              7]},
                                    scoring='roc_auc', verbose=3))])

In [34]:
random_search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [35]:
random_search.best_params_

{'min_child_weight': 1,
 'max_depth': 4,
 'learning_rate': 0.15,
 'gamma': 0.4,
 'colsample_bytree': 0.4}

In [36]:
main_model= XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=7, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None)
boss_model = ImblearnPipeline([
    ('preprocessor', preprocessor),
    ('classifier', main_model)
])

boss_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Overall', 'Age',
                                                   'Height(in cm)',
                                                   'Weight(in kg)',
                                                   'Weak Foot Rating',
                                                   'Skill Moves', 'Pace Total',
                                                   'Shooting Total',
                                                   'Passing Total',
                                                   'Dribbling Total',
                                                   'Defending Total',
                                                   'Physicality Total',
                                                   'Crossing', 'Finishing',
                                                   'Heading Accur...
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=10, max_leaves=None,
                               min_child_weight=7, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', predictor=None, ...))])

In [39]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
y_pred = boss_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate precision
precision = precision_score(y_test, y_pred, average = 'weighted')
print(f"Precision: {precision}")

# Calculate recall
recall = recall_score(y_test, y_pred, average = 'weighted')
print(f"Recall: {recall}")

# Calculate F1-score
f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f"F1 Score: {f1}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{conf_matrix}")


Accuracy: 0.9241280115066522
Precision: 0.9249930140653875
Recall: 0.9241280115066522
F1 Score: 0.9239852564001434
Confusion Matrix:
[[1897   90   68    0]
 [ 102 1763    1    0]
 [ 159    2  825    0]
 [   0    0    0  655]]


## Saving My Model

In [40]:
#Importing necessary models for saving model
import joblib

In [41]:
joblib.dump(boss_model, 'player_pred.joblib')

['player_pred.joblib']

In [42]:
player_position = joblib.load('player_pred.joblib')

## Testing my saved Model

In [106]:

testing = [91,34,185,81,4,4,80,88,83,87,39,78,75,92,90,89,82,79,78,92,72,87,79,82,82,63,89,84,90,5, 'France', 'Right']
len(testing)

32

In [107]:
test_arr = np.array(testing)
re = test_arr.reshape(1, -1)
stats = pd.DataFrame(re, columns = X.columns)

In [108]:
player_position.predict(stats)

array([2], dtype=int64)

In [105]:
X.loc[1]

Overall                   91
Age                       34
Height(in cm)            185
Weight(in kg)             81
Weak Foot Rating           4
Skill Moves                4
Pace Total                80
Shooting Total            88
Passing Total             83
Dribbling Total           87
Defending Total           39
Physicality Total         78
Crossing                  75
Finishing                 92
Heading Accuracy          90
Short Passing             89
Curve                     82
Acceleration              79
Agility                   78
Reactions                 92
Balance                   72
Shot Power                87
Jumping                   79
Stamina                   82
Strength                  82
Aggression                63
Vision                    89
Penalties                 84
Composure                 90
 GoalkeeperKicking         5
Nationality           France
Preferred Foot         Right
Name: 1, dtype: object